In [242]:
import re
from collections import namedtuple
from enum import Flag, auto, Enum, IntFlag
from string import ascii_letters
from typing import NamedTuple

from icecream import ic


class ValueType(IntFlag):
    INVALID = 0
    INT = auto()
    FLOAT = auto()
    NUMBER_LITERAL = auto()
    VARIABLE = auto()
    GROUPING = auto()
    OPERATOR = auto()

    @classmethod
    def float_type(cls):
        return cls.FLOAT | cls.NUMBER_LITERAL

    @classmethod
    def int_type(cls):
        return cls.INT | cls.NUMBER_LITERAL


GROUPING_CHARS = set('()')

MATH_CHARS = set('*/-+^')
EQUALITY_CHARS = set('=!<>')

EQUALITY_OPERATORS = {'==', '!=', '<', '>', '<=', '>='}
MATH_OPERATORS = {'*', '/', '-', '+', '^'}

ALL_NUMBER_LITERAL_CHARS = set('01234567890-+')
ALL_OPERATOR_CHARS = MATH_CHARS | EQUALITY_CHARS
ALL_VARIABLE_CHARS = set(ascii_letters) | {'_'}

ALL_OPERATORS = MATH_OPERATORS | EQUALITY_OPERATORS

RE_INT = re.compile(r'([-+]?)(\d+)')


class ValueWithIndexShift(NamedTuple):
    value: tuple[str]
    shift: int
    success: bool
    type: ValueType = ValueType.INVALID

    @property
    def islist(self):
        return isinstance(self.value, list)

    @property
    def isstr(self):
        return isinstance(self.value, str)

    @property
    def value_as_str(self):
        return ''.join(self.value)

    @classmethod
    def empty(cls):
        return cls(value=(), shift=0, success=False, type=ValueType.INVALID)




In [243]:
def is_variable_char(char):
    return char in ALL_VARIABLE_CHARS


def is_operator_sequence(data, i):
    peeked = peek_ahead(data, i, ALL_OPERATOR_CHARS.__contains__)
    return peeked.value_as_str in ALL_OPERATORS


def is_literal(data, i):
    value = peek_ahead(data, i, ALL_NUMBER_LITERAL_CHARS.__contains__)
    return value.success and RE_INT.match(value.value_as_str)

In [244]:
def _read_and_assign_type_if_success(data, i, predicate, type_if_success):
    value = peek_ahead(data, i, predicate)
    if value.success:
        return value._replace(type=type_if_success)
    return ValueWithIndexShift.empty()


def read_variable(data, index):
    return _read_and_assign_type_if_success(data, index, ALL_VARIABLE_CHARS.__contains__, ValueType.VARIABLE)


def read_operator(data, index):
    return _read_and_assign_type_if_success(data, index, ALL_OPERATOR_CHARS.__contains__, ValueType.OPERATOR)


def read_literal(data, index):
    value = _read_and_assign_type_if_success(data, index, ALL_NUMBER_LITERAL_CHARS.__contains__,
                                             ValueType.NUMBER_LITERAL)
    if value.success and RE_INT.match(value.value_as_str):
        return value

    return value.empty()


In [245]:
def peek_ahead(data, index, predicate):
    chars = []
    original_index = index
    while index < len(data) and predicate(data[index]):
        chars.append(data[index])
        index += 1
    return ValueWithIndexShift(value=tuple(chars), shift=index - original_index, success=bool(chars))


def handle_char(char, data, i):
    if is_variable_char(char):
        return read_variable(data, i)

    if is_literal(data, i):
        return read_literal(data, i)

    if is_operator_sequence(data, i):
        return read_operator(data, i)

    return ValueWithIndexShift.empty()


def parse_math(equation):
    groupings = []
    equation = list(re.sub(r'\s+', ' ', equation))
    i = 0
    while i < len(equation):
        char = equation[i]
        if char.isspace():
            i += 1
            continue
        ret = handle_char(char, equation, i)
        if not ret.success:
            i += 1
            continue

        groupings.append(ret)
        i += ret.shift

    return groupings


parse_math('x + y + -5 + 6 == 81')

[ValueWithIndexShift(value=('x',), shift=1, success=True, type=<ValueType.VARIABLE: 8>),
 ValueWithIndexShift(value=('+',), shift=1, success=True, type=<ValueType.OPERATOR: 32>),
 ValueWithIndexShift(value=('y',), shift=1, success=True, type=<ValueType.VARIABLE: 8>),
 ValueWithIndexShift(value=('+',), shift=1, success=True, type=<ValueType.OPERATOR: 32>),
 ValueWithIndexShift(value=('-', '5'), shift=2, success=True, type=<ValueType.NUMBER_LITERAL: 4>),
 ValueWithIndexShift(value=('+',), shift=1, success=True, type=<ValueType.OPERATOR: 32>),
 ValueWithIndexShift(value=('6',), shift=1, success=True, type=<ValueType.NUMBER_LITERAL: 4>),
 ValueWithIndexShift(value=('=', '='), shift=2, success=True, type=<ValueType.OPERATOR: 32>),
 ValueWithIndexShift(value=('8', '1'), shift=2, success=True, type=<ValueType.NUMBER_LITERAL: 4>)]